In [1]:
import torch
torch.set_default_tensor_type(torch.FloatTensor)
from torch.autograd import Variable
from torch.nn import functional as F
from torchmetrics import Accuracy, Recall, Precision, Specificity, ConfusionMatrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

from sklearn.metrics import precision_recall_curve, average_precision_score,roc_curve, auc, precision_score, recall_score, f1_score, confusion_matrix, accuracy_score

import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import matplotlib.pyplot as plt
from collections import Counter
import random
import math
import torch.optim as optim
from tabulate import tabulate
from ray import tune

from fractions import Fraction

import ray
ray.init(num_cpus=10)

2022-11-02 15:15:23,694	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8270 


Python version:,3.10.4
Ray version:,2.0.0
Dashboard:,http://127.0.0.1:8270


In [2]:
# Create Tensors to hold input and outputs.
#1,3,1+1,2,1
beta_1=np.append(np.array([4,4,-8,-4,6,8,-4,4,-6,-8]),np.zeros(90))
beta_2=np.append(np.array([-2,2,4,-2,3,-4,2,-2,-3,4]),np.zeros(90))
beta_3=np.append(np.array([-3,-3,6,-3,-4.5,-6,3,-3,4.5,6]),np.zeros(90))

beta_1=beta_1.reshape(-1,1)
beta_2=beta_2.reshape(-1,1)
beta_3=beta_3.reshape(-1,1)

n1=200
n2=200
n3=200
p=100
corval=0
prior=[1,2,3,4,5,11,12,13,14,15]

def sigmoid(X):
    return .5 * (1 + np.tanh(.5 * X))

def generate_data(corval,beta,n,p):
    mean=np.zeros(p)
    sigma=np.array([[corval**abs(i-j) for i in range(p)] for j in range(p)])
    x=torch.tensor(np.random.multivariate_normal(mean=mean,cov=sigma,size=n))
    Pi_test= sigmoid(x@beta)
    y=np.random.binomial(1,Pi_test.ravel(),n)
    x=x.to(torch.float32)
    #y=torch.tensor(y,dtype=torch.double)
    y=torch.tensor(y,dtype=torch.float)
    #y=y.to(torch.float32)
    return x,y
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

def cubic_root(x):
    return math.copysign(math.pow(abs(x), 1.0/3.0), x)

In [3]:
def train_model(config):
    split_num=5
    kf = KFold(n_splits=split_num)



    sum_test_BCE_loss=0
    for idx_train,idx_test in kf.split(inputs_1):
        lambda1,lr,ga= config["lambda1"],config["lr"],config["ga"]
        model_1=MLP(seed=1)
        model_2=MLP(seed=1)
        model_3=MLP(seed=1)
        max_iteration=1000
        learning_rate=lr
        params_to_optimize=list(model_1.parameters())+list(model_2.parameters())+list(model_3.parameters())
        optimizer = torch.optim.SGD(params_to_optimize, lr=learning_rate)
        loss_fn = torch.nn.BCELoss()
        scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,200,300,400,500,600,700,800], gamma=ga)

        BCE=[]

        for t in range(max_iteration):
            optimizer.zero_grad() # renew optimizer
            out_1= model_1(inputs_1[idx_train])
            out_2= model_2(inputs_2[idx_train])
            out_3= model_3(inputs_3[idx_train])# forward propagate

            # extract parameters
            #[:-1] for leaving out bias term#
            model_1_all_linear1_params= torch.cat([x.view(-1) for x in model_1.linear1.parameters()][:-1])
           

            model_2_all_linear1_params= torch.cat([x.view(-1) for x in model_2.linear1.parameters()][:-1])
    

            model_3_all_linear1_params= torch.cat([x.view(-1) for x in model_3.linear1.parameters()][:-1])
    


            # compute loss
            all_linear1_params=torch.cat(( model_1_all_linear1_params.reshape(1,-1),  model_2_all_linear1_params.reshape(1,-1), model_3_all_linear1_params.reshape(1,-1)),0)
            
            non_prior=np.setdiff1d([i for i in range(all_linear1_params.shape[1])],prior)
            
            BCE_loss_1 = loss_fn(out_1, targets_1[idx_train].reshape(-1,1))
            BCE_loss_2 = loss_fn(out_2, targets_2[idx_train].reshape(-1,1))
            BCE_loss_3 = loss_fn(out_3, targets_3[idx_train].reshape(-1,1))

#             l1_regularization = lambda1 * smooth_l1(all_linear1_params,prior)
            l1_regularization = lambda1 * torch.sum(torch.norm(all_linear1_params[:,non_prior],p=2,dim=0))

#             l2_regularization = lambda2* torch.sum(all_linear23_params.pow(2))
            BCE_loss=BCE_loss_1+BCE_loss_2+BCE_loss_3
            loss =BCE_loss+l1_regularization

            # record loss
            BCE.append(BCE_loss.item())

            # compute derivative
            loss.backward()

            # gradient descent
            optimizer.step()

            # learning rate decay
#             scheduler.step()
        
        
        test_out_1= model_1(train_x1[idx_test])
        test_out_2= model_2(train_x2[idx_test])
        test_out_3= model_3(train_x3[idx_test])
        
        test_BCE_loss_1 = loss_fn(test_out_1, targets_1[idx_test].reshape(-1,1))
        test_BCE_loss_2 = loss_fn(test_out_2, targets_2[idx_test].reshape(-1,1))
        test_BCE_loss_3 = loss_fn(test_out_3, targets_3[idx_test].reshape(-1,1))
        
        test_BCE_loss=test_BCE_loss_1+test_BCE_loss_2+test_BCE_loss_3
        
        sum_test_BCE_loss+=test_BCE_loss.item()

    #print("loss: ",)
    tune.report(my_test_BCE_loss=sum_test_BCE_loss/split_num)
     

In [4]:
def train_model_2(config):
    split_num=5
    kf = KFold(n_splits=split_num)
    
    
    lambda1,lr,eta= config["lambda1"], config["lr"],config["eta"]
    inputs_1,inputs_2,inputs_3=train_x1,train_x2,train_x3
    targets_1=(1-eta)*train_y1+eta*y_prior_1
    targets_2=(1-eta)*train_y2+eta*y_prior_2
    targets_3=(1-eta)*train_y3+eta*y_prior_3
    
    
    
    sum_tGM=0
    for idx_train,idx_test in kf.split(inputs_1):
       
        
        model_1=MLP(seed=1)
        model_2=MLP(seed=1)
        model_3=MLP(seed=1)
        max_iteration=1000
        learning_rate=lr
        params_to_optimize=list(model_1.parameters())+list(model_2.parameters())+list(model_3.parameters())
        optimizer = torch.optim.SGD(params_to_optimize, lr=learning_rate)
        loss_fn = torch.nn.BCELoss()
        scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,200,300,400,500,600,700,800], gamma=0.3)

        BCE=[]

        for t in range(max_iteration):
            optimizer.zero_grad() # renew optimizer
            out_1 = model_1(inputs_1[idx_train])
            out_2 = model_2(inputs_2[idx_train])
            out_3 = model_3(inputs_3[idx_train])# forward propagate

            # extract parameters
            model_1_all_linear1_params= torch.cat([x.view(-1) for x in model_1.linear1.parameters()][:-1])
           

            model_2_all_linear1_params= torch.cat([x.view(-1) for x in model_2.linear1.parameters()][:-1])
    

            model_3_all_linear1_params= torch.cat([x.view(-1) for x in model_3.linear1.parameters()][:-1])
    


            # compute loss

#             l1_regularization = lambda1 * smooth_l1(all_linear1_params,prior)
            
            all_linear1_params=torch.cat(( model_1_all_linear1_params.reshape(1,-1),  model_2_all_linear1_params.reshape(1,-1), model_3_all_linear1_params.reshape(1,-1)),0)
            
            non_prior=np.setdiff1d([i for i in range(all_linear1_params.shape[1])],prior)
            
            BCE_loss_1 = loss_fn(out_1, targets_1[idx_train].reshape(-1,1))
            BCE_loss_2 = loss_fn(out_2, targets_2[idx_train].reshape(-1,1))
            BCE_loss_3 = loss_fn(out_3, targets_3[idx_train].reshape(-1,1))
            
            l1_regularization = lambda1 * torch.sum(torch.norm(all_linear1_params[:,non_prior],p=2,dim=0))

            BCE_loss=BCE_loss_1+BCE_loss_2+BCE_loss_3
            loss =BCE_loss+l1_regularization

            # record loss
            BCE.append(BCE_loss.item())

            # compute derivative
            loss.backward()

            # gradient descent
            optimizer.step()

            # learning rate decay
            scheduler.step()
        
        
        test_out_1= model_1(train_x1[idx_test])
        test_out_2= model_2(train_x2[idx_test])
        test_out_3= model_3(train_x3[idx_test])
        
        test_prediction_1= (model_1(train_x1[idx_test])>0.5).clone().int()
        test_target_1= train_y1[idx_test].reshape(-1,1).int()

        test_prediction_2= (model_2(train_x2[idx_test])>0.5).clone().int()
        test_target_2= train_y2[idx_test].reshape(-1,1).int()

        test_prediction_3= (model_3(train_x3[idx_test])>0.5).clone().int()
        test_target_3= train_y3[idx_test].reshape(-1,1).int()

        test_prediction=np.append(test_prediction_1,test_prediction_2)
        test_prediction=np.append(test_prediction,test_prediction_3)
        test_prediction=test_prediction.tolist()

        test_target=np.append(test_target_1,test_target_2)
        test_target=np.append(test_target,test_target_3)
        test_target=test_target.tolist()

        final_matrix=confusion_matrix(test_target,test_prediction)
        tTPR=recall_score(test_target,test_target)#TPR
        tTNR=final_matrix[0,0]/(sum(final_matrix[0,:]))#TNR
        tGM=math.sqrt(tTPR*tTNR)

        sum_tGM+=tGM

    tune.report(my_test_sum_tGM=sum_tGM/split_num)
     

In [5]:
TPR=[]
TNR=[]
GM=[]
accuracy=[]
precision=[]
f1=[]=[]
vsSEN=[]
vsSPE=[]
vsGM=[]
vsMR=[]
vsCCR=[]


vs = np.zeros((1,100))

In [6]:
for i in range(10):
    setup_seed(i)
    x1,y1=generate_data(corval,beta_1,n1,p)
    x2,y2=generate_data(corval,beta_2,n2,p)
    x3,y3=generate_data(corval,beta_3,n3,p)

    train_x1, test_x1, train_y1, test_y1= train_test_split(x1,y1, test_size=0.2, random_state=1)
    train_x2, test_x2, train_y2, test_y2= train_test_split(x2,y2, test_size=0.2, random_state=1)
    train_x3, test_x3, train_y3, test_y3= train_test_split(x3,y3, test_size=0.2, random_state=1)
    
     # switch training set
    inputs_1,targets_1 = train_x1,train_y1
    inputs_2,targets_2 = train_x2,train_y2
    inputs_3,targets_3 = train_x3,train_y3

    # switching testing set
    test_inputs_1, test_targets_1 = test_x1,test_y1
    test_inputs_2, test_targets_2 = test_x2,test_y2
    test_inputs_3, test_targets_3 = test_x3,test_y3
    
################## first step######################################  
    prior=[1,2,3,4,5,11,12,13,14,15]

# choose lambda1,lambda2 by ray tune
    class MLP(torch.nn.Module):
        def __init__(self,seed):
            super(MLP, self).__init__()
            torch.manual_seed(seed)
            self.linear1= torch.nn.Linear(p,1)

        def forward(self, x):
            out = torch.sigmoid(self.linear1(x))
            return out

    config={
            "lr":tune.choice([0.6,0.7,0.8]),
            "lambda1": tune.choice([0.03,0.04,0.05]),
            "ga":tune.choice([0.3,0.4,0.5])
        }

    result = tune.run(
            train_model,  
            config=config,
            num_samples=30)
    
################## second step########################################    

    final_lambda=result.get_best_config('my_test_BCE_loss',mode='min')
    lambda1,lr,ga=final_lambda["lambda1"],final_lambda["lr"],final_lambda["ga"]
    learning_rate=lr
    max_iteration=1000
    
    BCE=[]
    
    model_1=MLP(seed=2)
    model_2=MLP(seed=3)
    model_3=MLP(seed=2)

    loss_fn = torch.nn.BCELoss()
    params_to_optimize=list(model_1.parameters())+list(model_2.parameters())+list(model_3.parameters())
    optimizer = torch.optim.SGD(params_to_optimize, lr=learning_rate)
    
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,200,300,400,500,600,700,800], gamma=ga)

    loss_record=[]
    
    for t in range(max_iteration):
    
    # renew optimizer
        optimizer.zero_grad()

        # forward propagate
        out_1= model_1(inputs_1)
        out_2= model_2(inputs_2)
        out_3= model_3(inputs_3)



        model_1_all_linear1_params= torch.cat([x.view(-1) for x in model_1.linear1.parameters()][:-1])


        model_2_all_linear1_params= torch.cat([x.view(-1) for x in model_2.linear1.parameters()][:-1])


        model_3_all_linear1_params= torch.cat([x.view(-1) for x in model_3.linear1.parameters()][:-1])


        # compute loss

        all_linear1_params=torch.cat(( model_1_all_linear1_params.reshape(1,-1),  model_2_all_linear1_params.reshape(1,-1), model_3_all_linear1_params.reshape(1,-1)),0)

        non_prior=np.setdiff1d([i for i in range(all_linear1_params.shape[1])],prior)

        BCE_loss_1 = loss_fn(out_1, targets_1.reshape(-1,1))
        BCE_loss_2 = loss_fn(out_2, targets_2.reshape(-1,1))
        BCE_loss_3 = loss_fn(out_3, targets_3.reshape(-1,1))




    #     l1_regularization = lambda1 * smooth_l1(all_linear1_params,prior)

        l1_regularization = lambda1 * torch.sum(torch.norm(all_linear1_params[:,non_prior],p=2,dim=0))
        BCE_loss=BCE_loss_1+BCE_loss_2+BCE_loss_3
        loss =BCE_loss+l1_regularization 

        loss_record.append(loss.item())
        # record loss
        BCE.append(BCE_loss.item())

        # compute derivative
        loss.backward()

        # gradient descent
        optimizer.step()

        # learning rate decay
        scheduler.step()

    
################## Y_prior##################        
    y_prior_1= (model_1(train_x1)).detach().squeeze()

    y_prior_2= (model_2(train_x2)).detach().squeeze()

    y_prior_3= (model_3(train_x3)).detach().squeeze()
    
    
################## third step##################

    #no prior
    prior=[]
    # choose lambda1,lambda2 by ray tune   
        
    class MLP(torch.nn.Module):
        def __init__(self,seed):
            super(MLP, self).__init__()
            torch.manual_seed(seed)
            self.linear1= torch.nn.Linear(p,1)

        def forward(self, x):
            out = torch.sigmoid(self.linear1(x))
            return out
        
        
    config={
        "lambda1": tune.choice([0.03,0.04,0.05]),
         "eta": tune.choice([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]),
         "lr":tune.choice([0.6,0.7,0.8])
                            }

    result = tune.run(
            train_model_2,  
            config=config,
            num_samples=100)
    

################## fourth step##################

    #no prior
    prior=[]
    #0.4,0.4
    # set hyperparamters
    final_parameter=result.get_best_config('my_test_sum_tGM',mode='max')
    lambda1,eta,lr=final_parameter["lambda1"],final_parameter["eta"],final_parameter["lr"]
    learning_rate=lr
    max_iteration=1000

    inputs_1,inputs_2,inputs_3=train_x1,train_x2,train_x3
    targets_1=(1-eta)*train_y1+eta*y_prior_1
    targets_2=(1-eta)*train_y2+eta*y_prior_2
    targets_3=(1-eta)*train_y3+eta*y_prior_3

    # switching testing set
    test_inputs_1, test_targets_1 = test_x1,test_y1
    test_inputs_2, test_targets_2 = test_x2,test_y2
    test_inputs_3, test_targets_3 = test_x3,test_y3

    # record loss descent
    BCE=[]


    # main nn object
    class MLP(torch.nn.Module):
        def __init__(self,seed):
            super(MLP, self).__init__()
            torch.manual_seed(seed)
            self.linear1= torch.nn.Linear(p,1)

        def forward(self, x):
            out = torch.sigmoid(self.linear1(x))
            return out



    model_1=MLP(seed=1)
    model_2=MLP(seed=2)
    model_3=MLP(seed=1)

    loss_fn = torch.nn.BCELoss()
    params_to_optimize=list(model_1.parameters())+list(model_2.parameters())+list(model_3.parameters())
    optimizer = torch.optim.SGD(params_to_optimize, lr=learning_rate)

    # learning rate decay scheme
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,200,300,400,500,600,700,800], gamma=0.3)

    loss_record=[]
    # loop for max_iteration times
    
    for t in range(max_iteration):
    
    # renew optimizer
        optimizer.zero_grad()

        # forward propagate
        out_1= model_1(inputs_1)
        out_2= model_2(inputs_2)
        out_3= model_3(inputs_3)



        # extract parameters
        #[:-1] for leaving out bias term#

        model_1_all_linear1_params= torch.cat([x.view(-1) for x in model_1.linear1.parameters()][:-1])


        model_2_all_linear1_params= torch.cat([x.view(-1) for x in model_2.linear1.parameters()][:-1])


        model_3_all_linear1_params= torch.cat([x.view(-1) for x in model_3.linear1.parameters()][:-1])

        # compute loss

        all_linear1_params=torch.cat(( model_1_all_linear1_params.reshape(1,-1),  model_2_all_linear1_params.reshape(1,-1), model_3_all_linear1_params.reshape(1,-1)),0)

        BCE_loss_1 = loss_fn(out_1, targets_1.reshape(-1,1))
        BCE_loss_2 = loss_fn(out_2, targets_2.reshape(-1,1))
        BCE_loss_3 = loss_fn(out_3, targets_3.reshape(-1,1))
        non_prior=np.setdiff1d([i for i in range(all_linear1_params.shape[1])],prior)
            #     l1_regularization = lambda1 * smooth_l1(all_linear1_params,prior)

        l1_regularization = lambda1 * torch.sum(torch.norm(all_linear1_params[:,non_prior],p=2,dim=0))

        BCE_loss=BCE_loss_1+BCE_loss_2+BCE_loss_3
        loss =BCE_loss+l1_regularization

        loss_record.append(loss.item())
        # record loss
        BCE.append(BCE_loss.item())

        # compute derivative
        loss.backward()

        # gradient descent
        optimizer.step()

        # learning rate decay
        scheduler.step()
    
####################################   prediction ####################################      

    prediction_1= (model_1(test_x1)>0.5).clone().int()
    target_1= test_y1.reshape(-1,1).int()

    prediction_2= (model_2(test_x2)>0.5).clone().int()
    target_2= test_y2.reshape(-1,1).int()

    prediction_3= (model_3(test_x3)>0.5).clone().int()
    target_3= test_y3.reshape(-1,1).int()

    prediction=np.append(prediction_1,prediction_2)
    prediction=np.append(prediction,prediction_3)
    prediction=prediction.tolist()

    target=np.append(target_1,target_2)
    target=np.append(target,target_3)
    target=target.tolist()

    final_matrix=confusion_matrix(target,prediction)
    TPR.append(recall_score(target,prediction))#TPR
    TNR.append(final_matrix[0,0]/(sum(final_matrix[0,:])))#TNR
    GM.append(math.sqrt(TPR[-1]*TNR[-1]))
    accuracy.append(accuracy_score(target,prediction))
    precision.append(precision_score(target,prediction))
    f1.append(f1_score(target,prediction))
    
    
####################################   variable selection ####################################     
    model_1_weight=model_1_all_linear1_params.detach().numpy().copy().reshape(1,-1)
    model_2_weight=model_1_all_linear1_params.detach().numpy().copy().reshape(1,-1)
    model_3_weight=model_1_all_linear1_params.detach().numpy().copy().reshape(1,-1)

    threshold=0.12
    mcl_w=np.zeros(p)

    for j in range(p):
        mcl_w[j]=cubic_root(model_1_weight[0,j]*model_2_weight[0,j]*model_3_weight[0,j])

    max_mcl_w=max(abs(mcl_w))
    for j in range(p):
        if (abs(mcl_w[j]))<=threshold*max_mcl_w:
            model_1_weight[0,j]=0
            model_2_weight[0,j]=0
            model_3_weight[0,j]=0
        else:
            model_1_weight[0,j]=model_1_weight[0,j]
            model_2_weight[0,j]=model_2_weight[0,j]
            model_3_weight[0,j]=model_3_weight[0,j]
    
    index=np.nonzero(model_1_weight)[1]
    
    vs[:,index]+=1
    
    beta=np.array(beta_1)
    trueindex=np.where(beta!=0)[0]
    trueindex=trueindex.tolist()
    trueindex=set(trueindex)
    TP=len(trueindex.intersection(index))
    FP=len(np.setdiff1d(index,trueindex))
    FN=len(trueindex)-TP
    TN=p-len(trueindex)-FP
    vsSEN.append(TP/(TP+FN))
    vsSPE.append(TN/(TN+FP))
    vsGM.append(math.sqrt(vsSEN[-1]*vsSPE[-1]))
    vsMR.append((FP+FN)/(TP+FN+TN+FP))
    vsCCR.append(1-vsMR[-1])
    
    print(i)
    

2022-11-02 15:15:27,177	WARNING function_trainable.py:619 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


Trial name,status,loc,ga,lambda1,lr,iter,total time (s),my_test_BCE_loss
train_model_2042d_00000,TERMINATED,10.224.0.12:15665,0.5,0.03,0.8,1,7.34891,0.688492
train_model_2042d_00001,TERMINATED,10.224.0.12:15720,0.3,0.05,0.6,1,7.50618,0.761505
train_model_2042d_00002,TERMINATED,10.224.0.12:15722,0.3,0.05,0.6,1,7.32661,0.761505
train_model_2042d_00003,TERMINATED,10.224.0.12:15724,0.4,0.03,0.8,1,7.29603,0.688492
train_model_2042d_00004,TERMINATED,10.224.0.12:15726,0.4,0.05,0.8,1,7.82434,0.763441
train_model_2042d_00005,TERMINATED,10.224.0.12:15728,0.4,0.04,0.7,1,7.28867,0.718945
train_model_2042d_00006,TERMINATED,10.224.0.12:15730,0.5,0.04,0.7,1,7.36754,0.718945
train_model_2042d_00007,TERMINATED,10.224.0.12:15731,0.4,0.04,0.6,1,7.21273,0.715083
train_model_2042d_00008,TERMINATED,10.224.0.12:15739,0.4,0.04,0.7,1,7.5274,0.718945
train_model_2042d_00009,TERMINATED,10.224.0.12:15741,0.3,0.04,0.7,1,7.24386,0.718945


(raylet) [2022-11-02 15:15:33,710 E 15218 15236] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_15-15-21_332017_13565 is over 95% full, available space: 326582272; capacity: 29394726912. Object creation will fail if spilling is required.


Result for train_model_2042d_00000:
  date: 2022-11-02_15-15-36
  done: false
  experiment_id: b6c2d11d05d7404f8b609c50175f18db
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 0.6884918332099914
  node_ip: 10.224.0.12
  pid: 15665
  time_since_restore: 7.348914861679077
  time_this_iter_s: 7.348914861679077
  time_total_s: 7.348914861679077
  timestamp: 1667373336
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 2042d_00000
  warmup_time: 0.003883838653564453
  
Result for train_model_2042d_00000:
  date: 2022-11-02_15-15-36
  done: true
  experiment_id: b6c2d11d05d7404f8b609c50175f18db
  experiment_tag: 0_ga=0.5000,lambda1=0.0300,lr=0.8000
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 0.6884918332099914
  node_ip: 10.224.0.12
  pid: 15665
  time_since_restore: 7.348914861679077
  time_this_iter_s: 7.348914861679077
  time_total_s: 7.348914861679077
  timestamp: 1667373336
  timesteps_since_restore: 0

Result for train_model_2042d_00004:
  date: 2022-11-02_15-15-40
  done: false
  experiment_id: 6401466b491d473e979014d9f26f9938
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 0.7634409308433533
  node_ip: 10.224.0.12
  pid: 15726
  time_since_restore: 7.824338674545288
  time_this_iter_s: 7.824338674545288
  time_total_s: 7.824338674545288
  timestamp: 1667373340
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 2042d_00004
  warmup_time: 0.005118846893310547
  
Result for train_model_2042d_00008:
  date: 2022-11-02_15-15-40
  done: false
  experiment_id: 76ccc01b4a414d33a1b25be893da66e7
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 0.7189445614814758
  node_ip: 10.224.0.12
  pid: 15739
  time_since_restore: 7.527404069900513
  time_this_iter_s: 7.527404069900513
  time_total_s: 7.527404069900513
  timestamp: 1667373340
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 2042d_00008
  war

...

Result for train_model_2_49f50_00097:
  date: 2022-11-02_15-31-59
  done: false
  experiment_id: 027ac7ef17624134ad0ed463aba8b80e
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_sum_tGM: 0.9703913992587099
  node_ip: 10.224.0.12
  pid: 25358
  time_since_restore: 5.479385137557983
  time_this_iter_s: 5.479385137557983
  time_total_s: 5.479385137557983
  timestamp: 1667374319
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 49f50_00097
  warmup_time: 0.00379180908203125
  
Result for train_model_2_49f50_00097:
  date: 2022-11-02_15-31-59
  done: true
  experiment_id: 027ac7ef17624134ad0ed463aba8b80e
  experiment_tag: 97_eta=0.2000,lambda1=0.0300,lr=0.8000
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_sum_tGM: 0.9703913992587099
  node_ip: 10.224.0.12
  pid: 25358
  time_since_restore: 5.479385137557983
  time_this_iter_s: 5.479385137557983
  time_total_s: 5.479385137557983
  timestamp: 1667374319
  timesteps_since_restore

2022-11-02 15:32:00,871	INFO tune.py:758 -- Total run time: 64.73 seconds (64.36 seconds for the tuning loop).


9


(raylet) [2022-11-02 15:32:04,581 E 15218 15236] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_15-15-21_332017_13565 is over 95% full, available space: 183918592; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 15:32:14,587 E 15218 15236] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_15-15-21_332017_13565 is over 95% full, available space: 183066624; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 15:32:24,591 E 15218 15236] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_15-15-21_332017_13565 is over 95% full, available space: 184717312; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 15:32:34,598 E 15218 15236] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_15-15-21_332017_13565 is over 95% full, available space: 183984128; capacity: 29394726912. Object creation will fail i

(raylet) [2022-11-02 15:37:14,777 E 15218 15236] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_15-15-21_332017_13565 is over 95% full, available space: 161935360; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 15:37:24,781 E 15218 15236] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_15-15-21_332017_13565 is over 95% full, available space: 160567296; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 15:37:34,787 E 15218 15236] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_15-15-21_332017_13565 is over 95% full, available space: 159752192; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 15:37:44,792 E 15218 15236] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_15-15-21_332017_13565 is over 95% full, available space: 159023104; capacity: 29394726912. Object creation will fail i

In [7]:
####################################   prediction ####################################   
from numpy import *
mean(TPR)
mean(TNR)
mean(GM)
mean(accuracy)
mean(precision)
mean(f1)


0.9247352966953131

0.9221361525311824

0.9231369494444491

0.9233333333333332

0.9230971877953362

0.9233454626879938

In [8]:
####################################   variable selection ####################################   
mean(vsSEN)
mean(vsSPE)
mean(vsGM)
mean(vsMR)
mean(vsCCR)

1.0

0.8855555555555557

0.9410319623744783

0.10300000000000001

0.897

...

(raylet) [2022-11-02 16:37:47,275 E 15218 15236] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_15-15-21_332017_13565 is over 95% full, available space: 24113152; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 16:37:57,283 E 15218 15236] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_15-15-21_332017_13565 is over 95% full, available space: 24051712; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 16:38:07,291 E 15218 15236] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_15-15-21_332017_13565 is over 95% full, available space: 24018944; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 16:38:17,299 E 15218 15236] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_15-15-21_332017_13565 is over 95% full, available space: 23973888; capacity: 29394726912. Object creation will fail if sp

(raylet) [2022-11-02 16:42:57,535 E 15218 15236] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_15-15-21_332017_13565 is over 95% full, available space: 22339584; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 16:43:07,544 E 15218 15236] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_15-15-21_332017_13565 is over 95% full, available space: 22310912; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 16:43:17,553 E 15218 15236] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_15-15-21_332017_13565 is over 95% full, available space: 22282240; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 16:43:27,561 E 15218 15236] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_15-15-21_332017_13565 is over 95% full, available space: 22163456; capacity: 29394726912. Object creation will fail if sp

(raylet) [2022-11-02 16:48:07,797 E 15218 15236] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_15-15-21_332017_13565 is over 95% full, available space: 20635648; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 16:48:17,806 E 15218 15236] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_15-15-21_332017_13565 is over 95% full, available space: 20574208; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 16:48:27,814 E 15218 15236] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_15-15-21_332017_13565 is over 95% full, available space: 20451328; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 16:48:37,822 E 15218 15236] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_15-15-21_332017_13565 is over 95% full, available space: 20439040; capacity: 29394726912. Object creation will fail if sp

(raylet) [2022-11-02 16:53:18,070 E 15218 15236] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_15-15-21_332017_13565 is over 95% full, available space: 14651392; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 16:53:28,078 E 15218 15236] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_15-15-21_332017_13565 is over 95% full, available space: 14544896; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 16:53:38,086 E 15218 15236] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_15-15-21_332017_13565 is over 95% full, available space: 14462976; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-02 16:53:48,094 E 15218 15236] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-02_15-15-21_332017_13565 is over 95% full, available space: 14368768; capacity: 29394726912. Object creation will fail if sp

2022-11-02 16:58:24,827	WARNING worker.py:1829 -- The log monitor on node rd-PowerEdge-R940xa failed with the following error:
Traceback (most recent call last):
  File "/home/rd/anaconda3/envs/envJK/lib/python3.10/site-packages/ray/_private/log_monitor.py", line 520, in <module>
    log_monitor.run()
  File "/home/rd/anaconda3/envs/envJK/lib/python3.10/site-packages/ray/_private/log_monitor.py", line 436, in run
    anything_published = self.check_log_files_and_publish_updates()
  File "/home/rd/anaconda3/envs/envJK/lib/python3.10/site-packages/ray/_private/log_monitor.py", line 339, in check_log_files_and_publish_updates
    file_info.reopen_if_necessary()
  File "/home/rd/anaconda3/envs/envJK/lib/python3.10/site-packages/ray/_private/log_monitor.py", line 79, in reopen_if_necessary
    new_inode = os.stat(self.filename).st_ino
FileNotFoundError: [Errno 2] No such file or directory: '/tmp/ray/session_2022-11-02_15-15-21_332017_13565/logs/worker-8d9fa64025061683c2f13efd7d06c33872d5059

In [9]:
vs

array([[10.,  8., 10.,  9.,  8., 10.,  9., 10., 10., 10.,  0.,  0.,  0.,
         1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
         0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.]])

(raylet) [2022-11-01 17:37:56,403 E 8943 8961] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-01_17-03-52_632605_8844 is over 95% full, available space: 484462592; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-01 17:38:06,413 E 8943 8961] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-01_17-03-52_632605_8844 is over 95% full, available space: 484425728; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-01 17:38:16,422 E 8943 8961] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-01_17-03-52_632605_8844 is over 95% full, available space: 484405248; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-01 17:38:26,432 E 8943 8961] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-01_17-03-52_632605_8844 is over 95% full, available space: 484388864; capacity: 29394726912. Object creation will fail if spilling i

(raylet) [2022-11-01 17:43:06,687 E 8943 8961] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-01_17-03-52_632605_8844 is over 95% full, available space: 483459072; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-01 17:43:16,694 E 8943 8961] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-01_17-03-52_632605_8844 is over 95% full, available space: 483446784; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-01 17:43:26,702 E 8943 8961] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-01_17-03-52_632605_8844 is over 95% full, available space: 483430400; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-01 17:43:36,710 E 8943 8961] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-01_17-03-52_632605_8844 is over 95% full, available space: 483418112; capacity: 29394726912. Object creation will fail if spilling i

(raylet) [2022-11-01 17:48:16,968 E 8943 8961] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-01_17-03-52_632605_8844 is over 95% full, available space: 482471936; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-01 17:48:26,977 E 8943 8961] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-01_17-03-52_632605_8844 is over 95% full, available space: 482455552; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-01 17:48:36,987 E 8943 8961] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-01_17-03-52_632605_8844 is over 95% full, available space: 482435072; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-01 17:48:46,996 E 8943 8961] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-01_17-03-52_632605_8844 is over 95% full, available space: 482414592; capacity: 29394726912. Object creation will fail if spilling i

(raylet) [2022-11-01 17:53:27,246 E 8943 8961] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-01_17-03-52_632605_8844 is over 95% full, available space: 481583104; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-01 17:53:37,256 E 8943 8961] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-01_17-03-52_632605_8844 is over 95% full, available space: 481570816; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-01 17:53:47,264 E 8943 8961] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-01_17-03-52_632605_8844 is over 95% full, available space: 481558528; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-01 17:53:57,272 E 8943 8961] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-01_17-03-52_632605_8844 is over 95% full, available space: 481517568; capacity: 29394726912. Object creation will fail if spilling i

In [7]:
i

9